# Создание Телеграм-ботов. Основы библиотеки pyTelegramBotAPI

На этом занятии мы разберемся, как писать код для телеграм-ботов: узнаем, что такое хэндлеры, разберемся со структурой сообщений в Телеграм и научимся отправлять пользователю различный контент. Помимо этого узнаем, как составлять цепочки сообщений и делать клавиатуры для ответов. В конце лекции мы с вами напишем интерфейс для программы VK Comment Toxic, которую вы писали на предыдущих занятиях.

---

Ссылки:
* Описание всех методов и классов Telegram Bot API: https://core.telegram.org/bots/api
* Краткая документация по pyTelegramBotAPI: https://pypi.org/project/pyTelegramBotAPI/
* Справочник по Telegram Bot API: https://tlgrm.ru/docs/bots/api
* Исходный код библиотеки pyTelegramBotAPI: https://github.com/eternnoir/pyTelegramBotAPI
* Гайд по созданию Телеграм-ботов №1: https://m.habr.com/ru/post/350648/
* Гайд по созданию Телеграм-ботов №2: https://mastergroosha.github.io/telegram-tutorial/

# Хэндлеры

В телеграм-ботах все построено вокруг ожидания сообщений от пользователя. Поэтому нет привычной структуры, когда есть какой-то основной код, который выполняется от начала до конца. Ну или на худой конец, когда есть бесконечный цикл, который периодически что-либо проверяет.  

Чтобы удобно реализовать ожидание сообщений, взаимодействие с ботом в библиотеке `pyTelegramBotAPI` реализуется через хэндлеры. Хэндлеры - это такие условия, которые библиотека сама постоянно проверяет. Если какое-то из условий выполняется - вызывается функция, которая идет за хэндлером.  

Функция может называться как угодно, но она не может возвращать значения. А еще у нее единственный аргумент - сообщение, которое нам отправил пользователь (и которое вызвало хэндлер)

In [ ]:
# Синтаксис использования хэндлеров

@bot.message_handler(content_types=['text'])          # хэндлер
def echo(message):                                    # функция, которую хэндлер вызывает
    bot.send_message(message.chat.id, message.text)

## Фильтры в хэндлерах

Основной хэндлер - `@bot.message_handler(filters)`. Этот хэндлер реагирует на сообщения пользователей.  

Его аргументы - это различные фильтры. По ним библиотека определяет, сработает данный хэндлер, или нет.

In [ ]:
# Реагирует на все сообщения с указанными в списке типами контента.
@bot.message_handler(content_types=['text', 'photo'])

# Типы контента:
# text, audio, document, photo, sticker,
# video, video_note, voice, location, contact, new_chat_members,
# left_chat_member, new_chat_title, new_chat_photo, delete_chat_photo,
# group_chat_created, supergroup_chat_created, channel_chat_created, 
# migrate_to_chat_id, migrate_from_chat_id, pinned_message

# Реагирует на все сообщения, подходящие под действие данного регулярного выражения.
# О регулярных выражениях - позже.
@bot.message_handler(regexp='SOME_REGEXP')

# Реагирует на все текстовые сообщения с указанными командами.
# Команды в сообщениях задаются так: '/start' или '/help'.
@bot.message_handler(commands=['start', 'help'])

# Реагирует на все сообщения, для которых функция func возвращает True
# У func единственный аргумент - сообщение
# func может быть задан как обычной функцией, так и лямбдой
@bot.message_handler(func=lambda message: 'проблема' in message.text)


# Логическое ИЛИ для хэндлеров
@bot.message_handler(commands=['hello'])
@bot.message_handler(func=lambda message: 'привет' in message.text)
def send_something(message):
    pass

# Логическое И для хэндлеров
@bot.message_handler(content_types=['text'], commands=['start', 'help'])

## Middleware хэндлер

Может изменить сообщение или какие-то переменные в боте до того, как сообщение перейдет к проверке остальными хэндлерами.

In [ ]:
telebot.apihelper.ENABLE_MIDDLEWARE = True  # Не забудьте включить использование middleware хэндлеров

@bot.middleware_handler(update_types=['message'])
def modify_message(bot_instance, message):
    # Модифицирует строку до того, как она попадет к остальным хэндлерам.
    message.another_text = message.text + ':changed'
    
# update_types - то, на что стриггерится хэндлер:
# message, edited_message, channel_post, edited_channel_post, inline_query,
# chosen_inline_result, callback_query, shipping_query, pre_checkout_query, poll

@bot.message_handler(commands=['start'])
def start(message):
    # Когда сообщение попадет сюда - оно уже изменено.
    assert message.another_text == message.text + ':changed'

# Структура `telebot.types.Message`

In [ ]:
{
    'content_type': 'text',        # Тип содержимого сообщения
    'message_id': 5,               # id сообщения
    'from_user':
    {
        'id': 333960329,           # id отправителя
        'first_name': 'Mikhail',
        'username': 'meshanya',
        'last_name': None
    },
    'date': 1520186598,
    'chat':
    {
        'type': 'private',
        'last_name': None,
        'first_name': 'Nybkox',
        'username': 'nybkox',
        'id': 333960329,           # id чата
        'title': None,
        'all_members_are_administrators': None
    },
    'forward_from_chat': None,
    'forward_from': None,
    'forward_date': None,
    'reply_to_message': None,
    'edit_date': None,
    'text': '/start',              # текст сообщения
    'entities': None,              # специальное содержимое (ссылки, хэштеги и т.д.)
    'audio': None,
    'document': None,
    'photo': None,
    'sticker': None,
    'video': None,
    'voice': None,
    'caption': None,
    'contact': None,
    'location': None,
    'venue': None,
    'new_chat_member': None,
    'left_chat_member': None,
    'new_chat_title': None,
    'new_chat_photo': None,
    'delete_chat_photo': None,
    'group_chat_created': None,
    'supergroup_chat_created': None,
    'channel_chat_created': None,
    'migrate_to_chat_id': None,
    'migrate_from_chat_id': None,
    'pinned_message': None
}

# Методы для отправки сообщений

Чаще всего нам понадобится использовать метод `bot.send_message()`. Разберемся подробнее с его аргументами.

* `chat_id` - id чата
* `text` - текст сообщения
* `disable_web_page_preview` - если `True`, то предпросмотр веб-страниц будет отключен.
* `reply_to_message_id` - id сообщения, на которое нужно ответить
* `reply_markup` - клавиатура для ответа (обсудим ниже)
* `parse_mode` - режим обработки текста (например, при значении `'HTML'` будет применять HTML-теги из текста)  

*`bot.send_message(message.chat.id, '<b>TEST</b>', parse_mode='HTML')`* -> **`TEST`**
* `disable_notification` - если `True`, то сообщение придет без звука

Все аргументы, кроме `chat_id` и `text` являются опциональными.

---

Также пробежимся по остальным методам для отправки контента.  

**ВАЖНО:** я указываю только аргументы, раскрывающие суть функции. На самом деле, у каждой функции есть еще несколько опциональных аргументов. Что они делают - вы сможете понять по названию. Чтобы узнать, какие аргументы есть у той или иной функции - введите ее название в PyCharm и посмотрите на всплывающую подсказку.

In [ ]:
# Пересылает сообщение
bot.forward_message(to_chat_id, from_chat_id, message_id)

# Все функции, отправляющие контент - могут отправлять его не только как файл, но и по id.
# Файл, единожды залитый на сервера Телеграма, получает id. В дальнейшем можно указывать id, а не повторно отправлять файл.
# id - это уникальная строка. Подробнее об этом поговорим в следующих лекциях.

# Отправляет фотографию
photo = open('/tmp/photo.png', 'rb')
bot.send_photo(chat_id, photo)
bot.send_photo(chat_id, "FILEID")
bot.send_photo(chat_id, "PHOTO_URL")  # Фотографии еще можно отправлять по ссылке

# Отправляет аудио
audio = open('/tmp/audio.mp3', 'rb')
bot.send_audio(chat_id, audio)
bot.send_audio(chat_id, "FILEID")

# Отправляет голосовое сообщение
voice = open('/tmp/voice.ogg', 'rb')
bot.send_voice(chat_id, voice)
bot.send_voice(chat_id, "FILEID")

# Отправляет документ
doc = open('/tmp/file.txt', 'rb')
bot.send_document(chat_id, doc)
bot.send_document(chat_id, "FILEID")

# Отправляет стикер
sti = open('/tmp/sti.webp', 'rb')
bot.send_sticker(chat_id, sti)
bot.send_sticker(chat_id, "FILEID")

# Отправляет видео
video = open('/tmp/video.mp4', 'rb')
bot.send_video(chat_id, video)
bot.send_video(chat_id, "FILEID")

# Отправляет видеозаметку
videonote = open('/tmp/videonote.mp4', 'rb')
bot.send_video_note(chat_id, videonote)
bot.send_video_note(chat_id, "FILEID")

# Отправляет местоположение
bot.send_location(chat_id=CHAT_ID, latitude=LATITUDE, longitude=LONGITUDE, live_period=LIVE_PERIOD)

# Отправляет состояние чата (например, "бот печатает").
# Можно ввести искуственную задержку, чтобы создать эффект реального набора
bot.send_chat_action(message.chat.id, 'typing')
time.sleep(3)

# Помимо 'typing' также бывают состояния:
# 'upload_photo', 'record_video', 'upload_video', 'record_audio',
# 'upload_audio', 'upload_document', 'find_location'


# Построение цепочек сообщений

Внутри функции можно указать, какая именно функция будет вызвана в ответ на следующее сообщение пользователя.  

Делается это командой `bot.register_next_step_handler(msg, next_func)`. Где `msg` - предыдущее в чате сообщение до ответа пользователя (смотрим и из сообщений бота, и из сообщений юзера), а `next_func` - это функция, которая будет вызвана.

---

Пример использования:

In [ ]:
@bot.message_handler(commands=['start'])
def say_hello(message):
    msg = bot.send_message(message.chat.id, 'Здравствуйте!\nВведите ваше имя')
    bot.register_next_step_handler(msg, ask_name)

def ask_name(message):
    bot.send_message(message.chat.id, 'Теперь я знаю, что вас зовут ' + message.text + '.')

# Создание клавиатур

In [ ]:
from telebot import types

# Клавиатура заполняется по одному элементу. В ряд помещается не более row_width кнопок.
markup = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True, one_time_keyboard=True)
itembtn1 = types.KeyboardButton('a')
itembtn2 = types.KeyboardButton('v')
itembtn3 = types.KeyboardButton('d')
markup.add(itembtn1, itembtn2, itembtn3)
bot.send_message(chat_id, "Choose one letter:", reply_markup=markup)

# Клавиатура заполняется рядами.
markup = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True, one_time_keyboard=True)
itembtna = types.KeyboardButton('a')
itembtnv = types.KeyboardButton('v')
itembtnc = types.KeyboardButton('c')
itembtnd = types.KeyboardButton('d')
itembtne = types.KeyboardButton('e')
markup.row(itembtna, itembtnv)
markup.row(itembtnc, itembtnd, itembtne)
bot.send_message(chat_id, "Choose one letter:", reply_markup=markup)

# Специальный тип клавиатуры, чтобы ее убрать.
markup = types.ReplyKeyboardRemove()
bot.send_message(chat_id, message, reply_markup=markup)

# Пользователя переводят в режим ответа на отправленное ботом сообщение.
markup = types.ForceReply()
bot.send_message(chat_id, "Send me another word:", reply_markup=markup)

# Прописываем стартовые команды `/`

Специальные клавиатуры могут появиться только в ответ на какое-то сообщение бота. При первом же запуске бота пользователю доступны только команды (пишутся так: `/start`). Кроме того, команды подразумевают переход бота в какой-то режим, а клавиатуры - дают варианты ответа.  

Как задать команды?

1. Открываем `@BotFather`.
2. Пишем команду `/setcommands`.
3. Прописываем команды для бота согласно подсказкам Отца Ботов.

---

Еще можем добавить описание бота, которое увидят новые пользователи. Для этого:

1. Открываем `@BotFather`.
2. Пишем команду `/setdescription`.
3. Прописываем описание бота согласно подсказкам Отца Ботов.

# Делаем интерфейс для Comment Toxic

Изменения:

* import vk
* import pyaspeller
* Изменил порядок получения логина и пароля
* Изменил процесс генерации комментария.

In [ ]:
# Жду код Артема.